### ENGENHARIA DE DADOS

---
Etapa de preparação da base de dados diária para modelagem estatística (SARIMAX), criação de  variáveis que capturem a dependência temporal (lags), sazonalidade semanal e anual , efeitos de calendário ,dinâmica recente das vendas.



In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('vendas_diarias.csv',parse_dates=["data"])

BASE DE DADOS

---



In [3]:
df.head(5)

,data,receita_diaria,qtd_diaria,ano,mes,dia_semana,fim_de_semana
0,2011-01-01,7156.5400,2.0,2011,1,5,1
1,2011-01-02,15012.1782,5.0,2011,1,6,1
2,2011-01-03,14313.0800,4.0,2011,1,0,0
3,2011-01-04,7855.6382,3.0,2011,1,1,0
4,2011-01-05,7855.6382,3.0,2011,1,2,0


In [4]:
df = df.sort_values("data").reset_index(drop=True) #organização temporal

### Criando features de calendário

---

 criando variáveis para  capturar padrões sazonais sem olhar para o futuro.

In [5]:
df["ano"] = df["data"].dt.year
df["mes"] = df["data"].dt.month
df["dia"] = df["data"].dt.day
df["dia_semana"] = df["data"].dt.dayofweek
df["semana_ano"] = df["data"].dt.isocalendar().week.astype(int)

df["fim_de_semana"] = df["dia_semana"].isin([5, 6]).astype(int)

## Criando variáveis dummies sazonais

---
vamos criar essas dummies para ser usadas como variáveis exógenas


In [6]:
df = pd.get_dummies(df, columns=["dia_semana"], drop_first=True) #dia da semana

In [7]:
df = pd.get_dummies(df, columns=["mes"], drop_first=True) #mes

## Criando Lags da variável alvo

---



In [9]:
#legs curtos
lags = [1, 7, 14, 30] #1 inércia diária , 7 padrão semanal , 14 e 30  efeito médio prazo.

for lag in lags:
    df[f"lag_{lag}"] = df["receita_diaria"].shift(lag)

In [11]:
#Lags da quantidade (variável explicativa) ajuda a explicar as variações da receita
for lag in lags:
    df[f"qtd_lag_{lag}"] = df["qtd_diaria"].shift(lag)

MÉDIA MOVEIS E DESVIO PADRÃO MÓVEL (VOLATILIDADE)

---
 Para detectar instabilidade , explicar picos e melhorar ML


In [12]:
windows = [7, 14, 30]

for w in windows:
    df[f"mm_{w}"] = df["receita_diaria"].rolling(w).mean()

Desvio padrão móvel (volatilidade)

In [13]:
for w in windows:
    df[f"std_{w}"] = df["receita_diaria"].rolling(w).std()

TRANSFORMAÇÃO LOGARITMICA PARA O SARIMAX

---
Modelamos no log e a previsão volta com `expm1`


In [14]:
df["receita_log"] = np.log1p(df["receita_diaria"])

In [16]:
df_model = df.dropna().reset_index(drop=True) #remoção de linhas com NA

In [17]:
df_model.shape

(1005, 39)

Agora a base está limpa para modelagem

---

## Separação train / test com split temporal

In [18]:
split_date = "2013-01-01"

train = df_model[df_model["data"] < split_date]
test = df_model[df_model["data"] >= split_date]

train.shape, test.shape

((701, 39), (304, 39))

## Definição de variáveis finais e variáveis exógenas para SARIMAX

---
Mantemos exógenas simples para evitar overfitting


In [19]:
TARGET = "receita_log" #target

In [20]:
exog_cols = [
    col for col in df_model.columns
    if col.startswith("dia_semana_")
    or col.startswith("mes_")
    or col.startswith("fim_de_semana")
]

In [21]:
exog_cols

['fim_de_semana',
 'dia_semana_1',
 'dia_semana_2',
 'dia_semana_3',
 'dia_semana_4',
 'dia_semana_5',
 'dia_semana_6',
 'mes_2',
 'mes_3',
 'mes_4',
 'mes_5',
 'mes_6',
 'mes_7',
 'mes_8',
 'mes_9',
 'mes_10',
 'mes_11',
 'mes_12']

SALVANDO AS BASES TRATADAS

---



In [ ]:
train.to_csv("train_model.csv", index=False)
test.to_csv("test_model.csv", index=False)